In [29]:
import numpy as np
import matplotlib.pyplot as plt
from wind2watts.fit.util import fit_to_df
import json

In [30]:
original = fit_to_df('../../data/strava_benchmark/Afternoon_Ride.fit')
no_power = fit_to_df('../../data/strava_benchmark/Afternoon_Ride_no_power.fit')

In [31]:
strava_data = json.load(open('../../data/strava_benchmark/watts_calc.json'))

In [33]:
pred_watts = np.array(strava_data['watts_calc'])

In [34]:
pred_watts_pos = pred_watts[pred_watts > 0.01]

In [ ]:
# Plot the original and strava side by side
fig, ax = plt.subplots(1, 2, figsize=(20, 10))
#ax[0].plot(original['power'], label='Original')
ax[1].plot(pred_watts_pos, label='Strava')
fig.legend()